In [1]:
from colmap import preprocess
from pathlib import Path
import cv2
import os 

In [2]:
IMAGE_ROOT = Path('data/sbd/6')
WORKING_DIR = Path('data/sbd_legacy/scan1')
WORKING_DIR.mkdir(exist_ok=True)
COLMAP_ROOT = Path(f'{WORKING_DIR}/colmap')
COLMAP_ROOT.mkdir(exist_ok=True)
WORKING_IMAGE_DIR = WORKING_DIR/'image'
WORKING_IMAGE_DIR.mkdir(exist_ok=True)


### 0. Process the Images

In [3]:
width = 1600
height = 1200

image_files = [x for x in IMAGE_ROOT.glob('*.JPG')]
for i,fname in enumerate(image_files):
    img = cv2.imread(str(fname))
    img = cv2.resize(img, (width, height))
    cv2.imwrite(str(WORKING_IMAGE_DIR/f'{i:06d}.png'), img)

#### RUN Colmap

In [4]:
feature_extractor_cmd = [
    'colmap', 'feature_extractor', 
    '--database_path', f'{COLMAP_ROOT}/database.db',
    '--image_path', str(WORKING_IMAGE_DIR),
    '--ImageReader.single_camera', '1',
    '--ImageReader.camera_model', 'SIMPLE_RADIAL',
]
os.system(' '.join(feature_extractor_cmd))

NameError: name 'os' is not defined

In [46]:
feature_matcher_cmd = [
    'colmap', 'exhaustive_matcher',
    '--database_path', f'{COLMAP_ROOT}/database.db',
]
os.system(' '.join(feature_matcher_cmd))


Exhaustive feature matching

Matching block [1/1, 1/1] in 2.387s
Elapsed time: 0.043 [minutes]


0

In [50]:
output_path = COLMAP_ROOT/'sparse'
output_path.mkdir(exist_ok=True)

mapper_cmd = [
    'colmap', 'mapper',
    '--database_path', f'{COLMAP_ROOT}/database.db',
    '--image_path', str(WORKING_IMAGE_DIR),
    '--output_path', f'{output_path}',
]
os.system(' '.join(mapper_cmd))



Loading database

Loading cameras... 1 in 0.000s
Loading matches... 509 in 0.002s
Loading images... 36 in 0.008s (connected 36)
Building correspondence graph... in 0.016s (ignored 0)

Elapsed time: 0.000 [minutes]


Finding good initial image pair


Initializing with image pair #8 and #10


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.450450e+03    0.00e+00    5.79e+05   0.00e+00   0.00e+00  1.00e+04        0    1.17e-03    2.85e-03
   1  1.089745e+03    1.36e+03    5.51e+05   0.00e+00   7.98e-01  1.27e+04        1    1.91e-03    4.77e-03
   2  7.499067e+02    3.40e+02    1.72e+04   2.86e+00   9.97e-01  3.81e+04        1    1.78e-03    6.56e-03
   3  7.471391e+02    2.77e+00    1.62e+04   1.59e+01   3.13e-01  3.62e+04        1    1.77e-03    8.33e-03
   4  7.387756e+02    8.36e+00    1.61e+04   1.59e+01   5.96e-01  3.65e+04        1    1.79e-03    1.01e-02
   5  7.310666e+02    7.71e+00   

0

In [56]:
cameras_bin = preprocess.read_cameras_binary(output_path/'0'/'cameras.bin')
images_bin = preprocess.read_images_binary(output_path/'0'/'images.bin')


In [67]:
import numpy as np
extrinsics = {}
for key, value in images_bin.items():
    R = preprocess.qvec2rotmat(value.qvec)
    t = value.tvec
    extrinsics[key] = np.concatenate(
            [np.hstack((R, t.reshape(3,1))),
            [[0,0,0,1]]])

In [68]:
extrinsics

{1: array([[-0.72122237,  0.61578662, -0.31724617, -0.18535695],
        [-0.54997872, -0.2305984 ,  0.80271276, -2.9119882 ],
        [ 0.42114332,  0.75341305,  0.50498226,  1.76552688],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 2: array([[-0.88265758,  0.40596165, -0.23687707, -0.67934953],
        [-0.38673391, -0.34086386,  0.85688314, -2.99077977],
        [ 0.26711886,  0.84794279,  0.4578652 ,  1.68834902],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 3: array([[-0.98175113, -0.14975111,  0.11721485, -0.47755145],
        [ 0.16643871, -0.37844875,  0.91053539, -3.16347442],
        [-0.09199388,  0.91342824,  0.39646687,  1.91911732],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 4: array([[-0.881664  , -0.36753507,  0.29595026, -0.32593002],
        [ 0.41205869, -0.29402999,  0.86241173, -3.24084035],
        [-0.22994831,  0.88230626,  0.41068168,  2.0082124 ],
        [ 0.        ,  0.        ,  0.        ,  1. 